# Criador Interativo de Plano de Corrida (Jack Daniels)

Notebook pensado para mapear o perfil do atleta de forma profunda, gerar zonas e simular um plano com o código do repositório. Use como vitrine do sistema ou como guia para preencher um novo perfil.


## Como funciona
1. Clona (ou confirma) o repositório atual.
2. Importa os módulos centrais (`user_profile`, `training_zones`, `plan_generator`).
3. Coleta informações-chave inspiradas no checklist Jack Daniels (objetivo, fisiologia, histórico, disponibilidade, lesões, preferências e rotina).
4. Calcula VDOT/zones e gera um plano-simulação com `PlanGenerator` para validar se os dados fazem sentido.
5. Exporta o perfil e o plano em JSON para reaproveitar depois.


In [ ]:
from pathlib import Path
import os
from pprint import pprint

repo_url = "https://github.com/tallesmedeiros/DecisionMaking.git"
repo_dir = Path.cwd()

# Verifica se já estamos dentro do repositório ou se precisamos clonar
if (repo_dir / 'plan_generator.py').exists():
    target_dir = repo_dir
    print('✅ Repositório já disponível neste diretório.')
elif (repo_dir / 'DecisionMaking').exists():
    target_dir = repo_dir / 'DecisionMaking'
    print('ℹ️ Pasta DecisionMaking já existe. Entrando nela.')
else:
    target_dir = repo_dir / 'DecisionMaking'
    print('⬇️ Clonando repositório com os arquivos necessários...')
    !git clone {repo_url}

os.chdir(target_dir)
print('📁 Diretório atual:', Path.cwd())
print('📄 Arquivos principais:')
!ls -1 *.py


In [ ]:
from datetime import date
import json
from typing import List, Dict

import ipywidgets as widgets
from IPython.display import display, clear_output

from user_profile import UserProfile, RaceGoal
from training_zones import TrainingZones, RaceTime
from plan_generator import PlanGenerator

print('Bibliotecas carregadas com sucesso.')


## Questionário guiado
Preencha as abas abaixo. Campos essenciais estão marcados, mas sinta-se livre para detalhar ao máximo para gerar planos mais fiéis.


In [ ]:
# Utilidades

def distance_to_km(label: str) -> float:
    mapping = {'5K': 5.0, '10K': 10.0, 'Half Marathon': 21.0975, 'Marathon': 42.195}
    return mapping.get(label, None)

# --- Seção 1: Objetivo e contexto ---
objective_widgets = {
    'name': widgets.Text(description='Nome da prova', placeholder='Ex.: Maratona de SP'),
    'distance': widgets.Dropdown(options=['5K', '10K', 'Half Marathon', 'Marathon'], description='Distância'),
    'date': widgets.DatePicker(description='Data'),
    'target_time': widgets.Text(description='Meta (HH:MM:SS)', placeholder='03:45:00'),
    'personal_best': widgets.Text(description='PB atual'),
    'motivation': widgets.Text(description='Motivação', placeholder='Performance / Saúde / Retomada'),
    'logistics': widgets.Textarea(description='Restrições', placeholder='Viagens, clima, terreno...'),
}
objective_box = widgets.VBox(list(objective_widgets.values()))

# --- Seção 2: Fisiologia ---
physiology_widgets = {
    'recent_distance': widgets.Dropdown(options=['5K', '10K', 'Half Marathon', 'Marathon'], description='Prova recente'),
    'recent_time': widgets.Text(description='Tempo recente', placeholder='00:45:30'),
    'hr_resting': widgets.IntText(description='FC repouso', value=0),
    'hr_max': widgets.IntText(description='FC máx', value=0),
    'age': widgets.IntText(description='Idade'),
    'weight_kg': widgets.FloatText(description='Peso (kg)'),
    'height_cm': widgets.FloatText(description='Altura (cm)'),
    'sex': widgets.Dropdown(options=['', 'M', 'F'], description='Sexo'),
}
physiology_box = widgets.VBox(list(physiology_widgets.values()))

# --- Seção 3: Histórico e consistência ---
history_widgets = {
    'years_running': widgets.FloatText(description='Anos correndo', value=0.0),
    'current_weekly_km': widgets.FloatText(description='Volume atual (km/sem)'),
    'average_weekly_km': widgets.FloatText(description='Média últimas 8-12 sem'),
    'recent_peak_weekly_km': widgets.FloatText(description='Pico recente (km)'),
    'consistent_days_per_week': widgets.IntSlider(description='Dias mantidos/sem', value=3, min=1, max=7),
    'tolerated_workouts': widgets.Textarea(description='Treinos tolerados', placeholder='Tempo run, intervalos, progressivo...'),
    'adherence_score': widgets.FloatSlider(description='Aderência prévia (%)', value=80, min=0, max=100),
    'experience_level': widgets.Dropdown(options=['beginner', 'intermediate', 'advanced'], description='Nível'),
}
history_box = widgets.VBox(list(history_widgets.values()))

# --- Seção 4: Disponibilidade ---
availability_widgets = {
    'days_per_week': widgets.IntSlider(description='Dias/semana alvo', value=4, min=1, max=7),
    'hours_per_day': widgets.FloatSlider(description='Tempo/sessão (h)', value=1.0, min=0.5, max=3.0, step=0.25),
    'preferred_time': widgets.Dropdown(options=['', 'morning', 'afternoon', 'evening'], description='Horário'),
    'preferred_location': widgets.SelectMultiple(options=['track', 'road', 'trail', 'treadmill'], description='Locais'),
    'weekly_schedule': widgets.Textarea(description='Grade semanal', placeholder='Seg: 6h-7h / Qua: 19h-20h ...'),
    'logistics': widgets.Textarea(description='Infra/acessos', placeholder='Pista às terças, academia...'),
}
availability_box = widgets.VBox(list(availability_widgets.values()))

# --- Seção 5: Lesões e prevenção ---
injury_widgets = {
    'previous_injuries': widgets.Textarea(description='Lesões prévias'),
    'current_injuries': widgets.Textarea(description='Lesões atuais'),
    'injury_triggers': widgets.Textarea(description='Gatilhos'),
    'red_zones': widgets.Textarea(description='Restrições'),
    'strength_routines': widgets.Textarea(description='Rotina de força'),
    'impact_limitations': widgets.Textarea(description='Limites de impacto'),
}
injury_box = widgets.VBox(list(injury_widgets.values()))

# --- Seção 6: Perfil psicológico ---
psych_widgets = {
    'rpe_tolerance': widgets.Dropdown(options=['baixa', 'moderada', 'alta'], description='Tolerância RPE'),
    'variety_preference': widgets.Dropdown(options=['prefere repetição', 'equilíbrio', 'prefere variedade'], description='Variedade'),
    'social_training': widgets.Text(description='Treino social', placeholder='grupos, parceiros...'),
}
psych_box = widgets.VBox(list(psych_widgets.values()))

# --- Seção 7: Vida profissional/familiar ---
life_widgets = {
    'stress_windows': widgets.Textarea(description='Janelas de stress', placeholder='Plantões, fechamento de mês...'),
    'sleep_constraints': widgets.Textarea(description='Sono/jet lag'),
    'family_constraints': widgets.Textarea(description='Compromissos familiares'),
}
life_box = widgets.VBox(list(life_widgets.values()))

# --- Seção 8: Clima e terreno ---
climate_widgets = {
    'race_weather': widgets.Text(description='Clima da prova', placeholder='Quente/úmido/frio'),
    'race_elevation': widgets.Text(description='Altimetria prova', placeholder='plano/ondulado'),
    'training_terrain': widgets.Text(description='Terreno de treino'),
}
climate_box = widgets.VBox(list(climate_widgets.values()))

# Acordeão com todas as seções
accordion = widgets.Accordion(children=[
    objective_box,
    physiology_box,
    history_box,
    availability_box,
    injury_box,
    psych_box,
    life_box,
    climate_box,
])
for i, title in enumerate([
    '1) Objetivo e contexto',
    '2) Fisiologia e VDOT',
    '3) Histórico e consistência',
    '4) Disponibilidade e logística',
    '5) Lesões e prevenção',
    '6) Perfil psicológico',
    '7) Vida profissional/familiar',
    '8) Clima/terreno da prova',
]):
    accordion.set_title(i, title)

display(accordion)


## Gerar perfil + plano de exemplo
Clique no botão para consolidar as respostas, estimar VDOT com Jack Daniels (se houver prova recente) e gerar um plano exemplo com `PlanGenerator`. Nada é enviado para APIs externas; tudo roda localmente para simular a experiência completa.


In [ ]:
output = widgets.Output()

save_button = widgets.Button(description='Gerar perfil e plano de exemplo', button_style='success', icon='check')


def parse_multiline(text: str) -> List[str]:
    return [item.strip() for item in text.split('\n') if item.strip()]


def build_profile(_: widgets.Button):
    with output:
        clear_output()
        # Main race goal
        main_goal = RaceGoal(
            distance=objective_widgets['distance'].value,
            date=objective_widgets['date'].value or date.today(),
            name=objective_widgets['name'].value,
            location='',
            is_main_goal=True,
            target_time=objective_widgets['target_time'].value or None,
        )

        profile = UserProfile(
            name='',
            age=physiology_widgets['age'].value or 0,
            weight_kg=physiology_widgets['weight_kg'].value or 0.0,
            height_cm=physiology_widgets['height_cm'].value or 0.0,
            gender=physiology_widgets['sex'].value,
            years_running=history_widgets['years_running'].value,
            current_weekly_km=history_widgets['current_weekly_km'].value,
            average_weekly_km=history_widgets['average_weekly_km'].value,
            recent_peak_weekly_km=history_widgets['recent_peak_weekly_km'].value,
            consistent_days_per_week=history_widgets['consistent_days_per_week'].value,
            tolerated_workouts=parse_multiline(history_widgets['tolerated_workouts'].value),
            adherence_score=history_widgets['adherence_score'].value,
            experience_level=history_widgets['experience_level'].value,
            main_race=main_goal,
            secondary_objectives=parse_multiline(objective_widgets['motivation'].value),
            days_per_week=availability_widgets['days_per_week'].value,
            hours_per_day=availability_widgets['hours_per_day'].value,
            preferred_time=availability_widgets['preferred_time'].value,
            preferred_location=list(availability_widgets['preferred_location'].value),
            weekly_schedule={'notes': availability_widgets['weekly_schedule'].value},
            recent_race_times={},
            zones_calculation_method='jack_daniels',
            hr_resting=physiology_widgets['hr_resting'].value or None,
            hr_max=physiology_widgets['hr_max'].value or None,
            previous_injuries=parse_multiline(injury_widgets['previous_injuries'].value),
            current_injuries=parse_multiline(injury_widgets['current_injuries'].value),
            injury_triggers=parse_multiline(injury_widgets['injury_triggers'].value),
            red_zones=parse_multiline(injury_widgets['red_zones'].value),
            available_equipment=parse_multiline(availability_widgets['logistics'].value),
            strength_routines=parse_multiline(injury_widgets['strength_routines'].value),
            impact_limitations=parse_multiline(injury_widgets['impact_limitations'].value),
        )

        # Race times and VDOT
        recent_dist_label = physiology_widgets['recent_distance'].value
        recent_time_str = physiology_widgets['recent_time'].value.strip()
        zones = None
        if recent_time_str:
            km = distance_to_km(recent_dist_label)
            if km:
                try:
                    zones = TrainingZones(method='jack_daniels')
                    zones.add_race_time('recent', RaceTime.from_time_string(km, recent_time_str))
                    zones.calculate_zones()
                    profile.recent_race_times[recent_dist_label] = recent_time_str
                    profile.vdot_estimate = zones.vdot
                except Exception as exc:
                    print('Não foi possível calcular VDOT:', exc)

        # Plano exemplo
        plan = PlanGenerator.generate_plan(
            name=f"Plano {main_goal.distance}",
            goal=main_goal.distance,
            level=history_widgets['experience_level'].value,
            days_per_week=availability_widgets['days_per_week'].value,
            training_zones=zones,
            user_profile=profile,
        )

        # Saídas
        print('📊 Perfil consolidado:')
        pprint(profile.to_dict(), sort_dicts=False)

        print()  # linha em branco
        print('🏃 Plano (resumo JSON):')
        plan_dict = plan.to_dict()
        print(json.dumps(plan_dict, indent=2))

        print()  # linha em branco
        print('💾 Dicas: salve o perfil para reuso com json.dump(profile.to_dict()) e regenere planos ajustando VDOT/testes.')

save_button.on_click(build_profile)

display(save_button, output)
